In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
#run_num = range(len(folder))
run_num = [0]
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]

__GAIN_TRAINING__ = True

real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx'], ['지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx'], ['용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx'], ['우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 202ms/step - gen_loss: 117.1975 - disc_loss: 0.7178 - rmse: 1.0900 - val_loss: 0.9516
Epoch 2/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 97.3222 - disc_loss: 0.5113 - rmse: 0.9413 - val_loss: 0.9747
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 92.5840 - disc_loss: 0.4552 - rmse: 0.9968 - val_loss: 0.8909
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 86.0059 - disc_loss: 0.4136 - rmse: 0.9949 - val_loss: 1.0657
Epoch 5/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 69.2039 - disc_loss: 0.3730 - rmse: 0.8471 -

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8494 - disc_loss: 0.3151 - rmse: 0.4049 - val_loss: 0.3754
Epoch 58/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.6207 - disc_loss: 0.3201 - rmse: 0.4721 - val_loss: 0.4073
Epoch 59/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.2800 - disc_loss: 0.3303 - rmse: 0.4042 - val_loss: 0.4254
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2385 - disc_loss: 0.3318 - rmse: 0.3914 - val_loss: 0.4153
Epoch 61/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.9543 - disc_loss: 0.3289 - rmse: 0.4278 - val_loss: 0.3713
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5274 - disc_loss: 0.3021 - rmse: 0.3564 - val_loss: 0.3620
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.9720 - disc_loss: 0.3049 - rmse: 0.4048 - val_loss: 0.3990
Epoch 64/2000
1/1 [===================

Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2480 - disc_loss: 0.2579 - rmse: 0.3218 - val_loss: 0.3089
Epoch 117/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.3926 - disc_loss: 0.2498 - rmse: 0.3140 - val_loss: 0.3251
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5126 - disc_loss: 0.2440 - rmse: 0.3279 - val_loss: 0.3990
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6789 - disc_loss: 0.2418 - rmse: 0.2910 - val_loss: 0.3066
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8529 - disc_loss: 0.2520 - rmse: 0.3108 - val_loss: 0.3205
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6523 - disc_loss: 0.2634 - rmse: 0.3788 - val_loss: 0.3303
Epoch 122/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8978 - disc_loss: 0.2546 - rmse: 0.3153 - val_loss: 0.3685
Epoch 123/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0761 - disc_loss: 0.2230 - rmse: 0.3294 - val_loss: 0.3082
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8294 - disc_loss: 0.2309 - rmse: 0.3403 - val_loss: 0.2985
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8572 - disc_loss: 0.2154 - rmse: 0.3346 - val_loss: 0.3121
Epoch 177/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.9789 - disc_loss: 0.2337 - rmse: 0.3172 - val_loss: 0.2900
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1032 - disc_loss: 0.2284 - rmse: 0.3780 - val_loss: 0.2981
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6680 - disc_loss: 0.2216 - rmse: 0.3052 - val_loss: 0.2904
Epoch 180/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3621 - disc_loss: 0.2167 - rmse: 0.2963 - val_loss: 0.2908
Epoch 181/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3134 - disc_loss: 0.2261 - rmse: 0.3095 - val_loss: 0.2958
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3598 - disc_loss: 0.2236 - rmse: 0.2699 - val_loss: 0.2819
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2020 - disc_loss: 0.2202 - rmse: 0.2633 - val_loss: 0.3416
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7664 - disc_loss: 0.2058 - rmse: 0.2594 - val_loss: 0.3715
Epoch 236/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2218 - disc_loss: 0.2208 - rmse: 0.3178 - val_loss: 0.2934
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6951 - disc_loss: 0.2179 - rmse: 0.2935 - val_loss: 0.2790
Epoch 238/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6417 - disc_loss: 0.2009 - rmse: 0.3206 - val_loss: 0.3351
Epoch 239/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1975 - disc_loss: 0.1870 - rmse: 0.2446 - val_loss: 0.2955
Epoch 871/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4755 - disc_loss: 0.1783 - rmse: 0.2254 - val_loss: 0.3081
Epoch 872/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8693 - disc_loss: 0.1682 - rmse: 0.2355 - val_loss: 0.2724
Epoch 873/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6163 - disc_loss: 0.1679 - rmse: 0.2828 - val_loss: 0.2340
Epoch 874/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4985 - disc_loss: 0.1740 - rmse: 0.2807 - val_loss: 0.3132
Epoch 875/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1336 - disc_loss: 0.1804 - rmse: 0.2411 - val_loss: 0.2590
Epoch 876/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1471 - disc_loss: 0.1708 - rmse: 0.2221 - val_loss: 0.2727
Epoch 877/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4129 - disc_loss: 0.1728 - rmse: 0.2248 - val_loss: 0.2230
Epoch 929/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.2059 - disc_loss: 0.2022 - rmse: 0.2746 - val_loss: 0.2451
Epoch 930/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0441 - disc_loss: 0.1688 - rmse: 0.2197 - val_loss: 0.2292
Epoch 931/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7824 - disc_loss: 0.1950 - rmse: 0.2417 - val_loss: 0.2577
Epoch 932/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.6564 - disc_loss: 0.1822 - rmse: 0.2353 - val_loss: 0.2359
Epoch 933/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7562 - disc_loss: 0.1783 - rmse: 0.2405 - val_loss: 0.2541
Epoch 934/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5562 - disc_loss: 0.1765 - rmse: 0.2790 - val_loss: 0.2301
Epoch 935/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2822 - disc_loss: 0.1680 - rmse: 0.2416 - val_loss: 0.2350
Epoch 987/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.7185 - disc_loss: 0.1682 - rmse: 0.2464 - val_loss: 0.2627
Epoch 988/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4649 - disc_loss: 0.1790 - rmse: 0.2278 - val_loss: 0.2512
Epoch 989/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4592 - disc_loss: 0.1761 - rmse: 0.2259 - val_loss: 0.2545
Epoch 990/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9929 - disc_loss: 0.1727 - rmse: 0.2264 - val_loss: 0.2501
Epoch 991/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0744 - disc_loss: 0.1784 - rmse: 0.2403 - val_loss: 0.2312
Epoch 992/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9435 - disc_loss: 0.1747 - rmse: 0.2131 - val_loss: 0.2470
Epoch 993/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7910 - disc_loss: 0.1803 - rmse: 0.2167 - val_loss: 0.2271
Epoch 1045/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0248 - disc_loss: 0.1747 - rmse: 0.2253 - val_loss: 0.2241
Epoch 1046/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5350 - disc_loss: 0.1635 - rmse: 0.2361 - val_loss: 0.2229
Epoch 1047/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8656 - disc_loss: 0.1680 - rmse: 0.2183 - val_loss: 0.2269
Epoch 1048/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6445 - disc_loss: 0.1632 - rmse: 0.2064 - val_loss: 0.2354
Epoch 1049/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7922 - disc_loss: 0.1794 - rmse: 0.2193 - val_loss: 0.2468
Epoch 1050/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6861 - disc_loss: 0.1753 - rmse: 0.2335 - val_loss: 0.2164
Epoch 1051/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5148 - disc_loss: 0.1673 - rmse: 0.2590 - val_loss: 0.2331
Epoch 1103/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.8339 - disc_loss: 0.1803 - rmse: 0.2188 - val_loss: 0.2748
Epoch 1104/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5191 - disc_loss: 0.1592 - rmse: 0.2524 - val_loss: 0.2690
Epoch 1105/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2064 - disc_loss: 0.1716 - rmse: 0.2439 - val_loss: 0.2430
Epoch 1106/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8728 - disc_loss: 0.1707 - rmse: 0.2154 - val_loss: 0.2260
Epoch 1107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3740 - disc_loss: 0.1821 - rmse: 0.2261 - val_loss: 0.2161
Epoch 1108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4913 - disc_loss: 0.1687 - rmse: 0.2537 - val_loss: 0.2224
Epoch 1109/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [59]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)



(26304, 48)
tp


In [60]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 48) val_df.shape :  (5261, 48) test_df.shape: (2631, 48)


In [61]:
#fake_df.shape[0]/8760

In [62]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [63]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [64]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong_pull/phosphorus/


In [65]:
#rnn_epochs, rnn_in_setps
__RNN_TRAINING__ = True
rnn_continue_train = False
rnn_epochs = 15
rnn_steps_per_epoch = 5

## 모델 학습

In [66]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
    epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

Epoch 1/15
5/5 [==============================] - 5s 1s/step - loss: 0.8719 - mean_absolute_error: 0.7020 - nse: 0.0174 - val_loss: 0.8058 - val_mean_absolute_error: 0.6422 - val_nse: 0.1293
Epoch 2/15
5/5 [==============================] - 5s 1s/step - loss: 0.6650 - mean_absolute_error: 0.5999 - nse: 0.2671 - val_loss: 0.7249 - val_mean_absolute_error: 0.6094 - val_nse: 0.2137
Epoch 3/15
5/5 [==============================] - 5s 1s/step - loss: 0.5130 - mean_absolute_error: 0.5179 - nse: 0.4296 - val_loss: 0.5476 - val_mean_absolute_error: 0.5143 - val_nse: 0.4060
Epoch 4/15
5/5 [==============================] - 5s 1s/step - loss: 0.4362 - mean_absolute_error: 0.4844 - nse: 0.4779 - val_loss: 0.5531 - val_mean_absolute_error: 0.5267 - val_nse: 0.3970
Epoch 5/15
5/5 [==============================] - 5s 1s/step - loss: 0.3792 - mean_absolute_error: 0.4427 - nse: 0.5312 - val_loss: 0.4867 - val_mean_absolute_error: 0.4847 - val_nse: 0.4701
Epoch 6/15
5/5 [=============================

In [67]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/yeong/models/yeong_pull/phosphorus/
year : 2016 ~ 2018
run :  [0]
target :  tp   6
length :  26304
train=7, val=1, test=2
all_nse :  [0.49487227]
all_pbias :  [2.1311557]
all_mae :  0.017530765
all_rmse :  0.025366694
all_Rs :  0.4978190752564302
all_R :  0.70556295


In [68]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.5606299]
train_pbias :  [1.1484615]
train_mae :  0.016588898
train_rmse :  0.023891956
train_Rs :  0.5613340227430399
train_R :  0.7492223


In [69]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [-0.01045656]
val_pbias :  [4.690815]
val_mae :  0.020184599
val_rmse :  0.030463153
val_Rs :  0.131732765358322
val_R :  0.3629501


In [70]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.42237443]
test_pbias :  [6.0136995]
test_mae :  0.020253198
test_rmse :  0.028563246
test_Rs :  0.4449918895226155
test_R :  0.6670771


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

